In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import numpy as np
import math

# Download all data

In [2]:
url = "http://www.revenue.state.il.us/LocalGovernment/Disbursements/SalesRelated/"
response = requests.get(url)
html = BeautifulSoup(response.text, "lxml")

In [3]:
download_links = []
for i in html.find_all("a"):
    href = i.get("href")
    if "sales" in href:
        download_links.append(url+href)
with open("download_links", "w+") as fd:
    for i in download_links:
        fd.write(i+"\n")

In [4]:
!wget -i download_links -P data/

--2018-06-26 14:50:34--  http://www.revenue.state.il.us/LocalGovernment/Disbursements/SalesRelated/sales07160617.xlsx
Resolving www.revenue.state.il.us (www.revenue.state.il.us)... 163.191.201.132
Connecting to www.revenue.state.il.us (www.revenue.state.il.us)|163.191.201.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308546 (301K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘data/sales07160617.xlsx.1’

sales07160617.xlsx. 100%[===================>] 301.31K   161KB/s    in 1.9s    

2018-06-26 14:50:36 (161 KB/s) - ‘data/sales07160617.xlsx.1’ saved [308546/308546]

--2018-06-26 14:50:36--  http://www.revenue.state.il.us/LocalGovernment/Disbursements/SalesRelated/sales07150616.xlsx
Reusing existing connection to www.revenue.state.il.us:80.
HTTP request sent, awaiting response... 200 OK
Length: 303374 (296K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘data/sales07150616.xlsx.1’

sales0715

# Convert pdf to csv
I use tabula, a java package, to grab table in a pdf file and convert it to csv.

However, due to the scheme of the original file, we need to deal with the rows and columns. Before 2013, month information is not in onw row. 

As you can see in the following comparison between the original csv and output, the framework of the data has been changed.

For further analysis, I add a new column, "Span", which distinguishes the row with same element.

In [6]:
# Run this command in terminal in directory data/
# This script uses tabula to grab table in a pdf file and convert it to csv
!./pdf2csv.sh

/bin/sh: 1: ./pdf2csv.sh: not found


In [7]:
new_csvs = []
for i in os.listdir("data/"):
    if ".csv" in i:
        new_csvs.append(i)
# Deal with rows and cols
for csv in new_csvs:
    span = csv[5:13] # indicates the span this csv covers
    df = pd.read_csv("data/"+csv)
    df = df.drop(range(60,len(df),61), axis=0) # remove unnecessary rows eg:repeated scheme
    df = df.reset_index(drop=True)
    # Rename columns
    df.columns = ["LocalGovernment", "Tax", "Vendor#", "Temp", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC", "FY_total"]
    # The following is to move Jan-Jun to the same row as Jul-Dec
    jan2jun = df.iloc[1::2, 4:].reset_index(drop=True)
    jan2jun.columns = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "FY_total"]
    df = df.drop(range(1, len(df), 2), axis=0).reset_index(drop=True)
    for i in jan2jun.columns:
        df[i] = jan2jun[:][i]
    df = df.drop("Temp", axis=1).reset_index(drop=True)
    df = df.dropna(how="all")
    df["Span"] = np.array([span]*len(df))
    # tax data should be float type
    for i in range(3, 16):
        df.iloc[:, i] = df.iloc[:, i].map(lambda x: float(x.replace(",", "")))
    df.iloc[:, 2] = df.iloc[:, 2].map(lambda x: str(int(x)) if not math.isnan(x) else x) # Vender# should be string
    df.to_csv("sales"+span, index=False)

In [8]:
# Original csv
pd.read_csv("data/sales07100611.pdf.csv")

,Local GovernmentTaxVendor #FY Total,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,ABINGDON,MT,390000010.0,JUL - DEC,"13,641.08","17,114.13","16,600.83","15,020.00","13,557.00","15,437.33",NaN
1,NaN,NaN,NaN,JAN - JUN,"15,165.40","13,950.87","16,010.03","12,447.11","13,731.71","15,527.35","178,202.84"
2,ADAMS COUNTY GOVERNMENT,CST,290000001.0,JUL - DEC,"150,484.34","168,827.75","183,551.91","165,368.23","154,026.52","158,484.38",NaN
3,NaN,NaN,NaN,JAN - JUN,"203,475.61","185,012.67","219,194.07","133,647.28","146,933.53","164,238.97","2,033,245.26"
4,ADAMS COUNTY GOVERNMENT,CT,290000001.0,JUL - DEC,"46,403.85","29,058.04","46,075.88","44,608.86","41,027.63","47,996.68",NaN
5,NaN,NaN,NaN,JAN - JUN,"181,700.29","42,695.32","44,107.32","29,748.19","30,176.41","40,821.14","624,419.61"
6,ADDIEVILLE,MT,390000018.0,JUL - DEC,"3,288.19","3,201.13","4,850.22","3,495.84","3,228.25","3,039.54",NaN
7,NaN,NaN,NaN,JAN - JUN,"3,074.30","4,748.87","2,663.88","1,024.98","4,048.33","3,709.58","40,373.11"
8,ADDISON,HMR,390000029.0,JUL - DEC,"367,251.20","366,446.79","410,460.87","384,395.38","370,117.31","378,246.27",NaN
9,NaN,NaN,NaN,JAN - JUN,"397,488.00","385,376.73","454,824.86","334,609.25","317,153.72","393,603.73","4,559,974.11"


In [9]:
# Output csv
df

,LocalGovernment,Tax,Vendor#,JUL,AUG,SEP,OCT,NOV,DEC,FY_total,JAN,FEB,MAR,APR,MAY,JUN,Span
0,ABINGDON,MT,390000010,1.362733e+04,1.540835e+04,1.427931e+04,1.426229e+04,1.493429e+04,1.487291e+04,1.712648e+05,1.276260e+04,1.377931e+04,1.631620e+04,1.298607e+04,1.284884e+04,1.518732e+04,07070608
1,ADAMS COUNTY GOVERNMENT,CST,290000001,1.524827e+05,1.564271e+05,1.646141e+05,1.557228e+05,1.566063e+05,1.537633e+05,1.900610e+06,1.516090e+05,1.652202e+05,2.022498e+05,1.433219e+05,1.405145e+05,1.580784e+05,07070608
2,ADAMS COUNTY GOVERNMENT,CT,290000001,4.181071e+04,5.018188e+04,4.328123e+04,5.969363e+04,4.195395e+04,4.736019e+04,5.645423e+05,4.516106e+04,4.009830e+04,5.628823e+04,4.712445e+04,5.204015e+04,3.954848e+04,07070608
3,ADDIEVILLE,MT,390000018,2.915070e+03,3.072780e+03,4.085920e+03,3.518130e+03,3.631550e+03,4.339260e+03,4.095379e+04,5.359610e+03,4.279540e+03,3.374160e+03,2.203930e+03,1.657040e+03,2.516800e+03,07070608
4,ADDISON,HMR,390000029,3.798454e+05,4.106630e+05,4.298512e+05,3.874225e+05,4.140372e+05,4.238580e+05,4.874356e+06,4.194947e+05,4.503458e+05,4.370215e+05,3.755154e+05,3.734915e+05,3.728097e+05,07070608
5,ADDISON,MT,390000029,5.882272e+05,6.311999e+05,6.583750e+05,6.052985e+05,6.429038e+05,6.620651e+05,7.597320e+06,6.895270e+05,6.546598e+05,6.940824e+05,5.853606e+05,5.920834e+05,5.935374e+05,07070608
6,ADDISON,MTART,390000029,2.757000e+01,1.174000e+01,1.086000e+01,1.781000e+01,1.476000e+01,1.781000e+01,2.452600e+02,1.973000e+01,2.165000e+01,2.165000e+01,2.165000e+01,2.165000e+01,3.838000e+01,07070608
7,ADELINE,MT,390000030,6.016000e+01,1.056000e+02,6.340000e+00,1.923200e+02,3.440000e+02,1.675200e+02,2.170500e+03,2.949600e+02,2.249600e+02,1.976100e+02,1.272400e+02,2.340800e+02,2.157100e+02,07070608
8,ALBANY,MT,390000020,4.251870e+03,3.616280e+03,5.085790e+03,4.189670e+03,4.384450e+03,5.419110e+03,4.603387e+04,3.069570e+03,2.848560e+03,3.366190e+03,3.216130e+03,2.993470e+03,3.592780e+03,07070608
9,ALBERS,MT,390000045,5.650210e+03,5.359150e+03,5.380030e+03,4.761000e+03,5.054500e+03,4.489340e+03,6.273062e+04,5.731660e+03,4.925670e+03,6.345130e+03,4.702250e+03,4.736550e+03,5.595130e+03,07070608


In [10]:
df.dtypes

LocalGovernment     object
Tax                 object
Vendor#             object
JUL                float64
AUG                float64
SEP                float64
OCT                float64
NOV                float64
DEC                float64
FY_total           float64
JAN                float64
FEB                float64
MAR                float64
APR                float64
MAY                float64
JUN                float64
Span                object
dtype: object

# Deal with xlsx file
In 11-12 and 12-13, month infomation is not in the same row. So I try to manipulate them.

In [11]:
# Deal with 07110612
df = pd.read_excel("data/sales07110612.xls", skiprows=5)
span = "07110612"
df.columns = ["LocalGovernment", "Tax", "Vendor#", "Temp", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC", "FY_total"]
# The following is to move Jan-Jun to the same row as Jul-Dec
jan2jun = df.iloc[1::2, 4:].reset_index(drop=True)
jan2jun.columns = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "FY_total"]
df = df.drop(range(1, len(df), 2), axis=0).reset_index(drop=True)
for i in jan2jun.columns:
    df[i] = jan2jun[:][i]
df = df.drop("Temp", axis=1).reset_index(drop=True)
df = df.dropna(how="all")
df["Span"] = np.array([span]*len(df))
df.iloc[:, 2] = df.iloc[:, 2].map(lambda x: str(int(x)) if not math.isnan(x) else x) # Vender# should be string
df.to_csv("sales"+span, index=False)

In [12]:
# Deal with 07120613
df = pd.read_excel("data/sales07120613.xlsx", skiprows=5)
span = "07120613"
df.columns = ["LocalGovernment", "Tax", "Vendor#", "Temp", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC", "FY_total"]
# The following is to move Jan-Jun to the same row as Jul-Dec
jan2jun = df.iloc[1::2, 4:].reset_index(drop=True)
jan2jun.columns = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "FY_total"]
df = df.drop(range(1, len(df), 2), axis=0).reset_index(drop=True)
for i in jan2jun.columns:
    df[i] = jan2jun[:][i]
df = df.drop("Temp", axis=1).reset_index(drop=True)
df = df.dropna(how="all")
df["Span"] = np.array([span]*len(df))
df.iloc[:, 2] = df.iloc[:, 2].map(lambda x: str(int(x)) if not math.isnan(x) else x) # Vender# should be string
df.to_csv("sales"+span, index=False)

In [13]:
df.dtypes

LocalGovernment     object
Tax                 object
Vendor#             object
JUL                float64
AUG                float64
SEP                float64
OCT                float64
NOV                float64
DEC                float64
FY_total           float64
JAN                float64
FEB                float64
MAR                float64
APR                float64
MAY                float64
JUN                float64
Span                object
dtype: object

In [14]:
df

,LocalGovernment,Tax,Vendor#,JUL,AUG,SEP,OCT,NOV,DEC,FY_total,JAN,FEB,MAR,APR,MAY,JUN,Span
0,ABINGDON,MT,390000010,1.722710e+04,1.779580e+04,1.654137e+04,1.579451e+04,1.514512e+04,1.517729e+04,1.865800e+05,1.449995e+04,1.478162e+04,1.913205e+04,1.246071e+04,1.244261e+04,1.558189e+04,07120613
1,ADAMS COUNTY GOVERNMENT,CST,290000001,1.626250e+05,1.755619e+05,1.751585e+05,1.633485e+05,1.755020e+05,1.638272e+05,2.071669e+06,1.704899e+05,1.839615e+05,2.205799e+05,1.536350e+05,1.521091e+05,1.748706e+05,07120613
2,ADAMS COUNTY GOVERNMENT,CT,290000001,4.704802e+04,4.091625e+04,4.012886e+04,3.741732e+04,4.828498e+04,4.793488e+04,5.331234e+05,5.378588e+04,4.972298e+04,5.347450e+04,3.881460e+04,3.446861e+04,4.112649e+04,07120613
3,ADDIEVILLE,MT,390000018,4.017390e+03,5.435230e+03,3.593360e+03,5.558310e+03,4.136270e+03,3.937480e+03,4.930032e+04,4.819890e+03,4.204660e+03,3.196030e+03,3.231100e+03,3.771300e+03,3.399300e+03,07120613
4,ADDISON,HMR,390000029,5.115589e+05,5.468698e+05,5.343887e+05,5.277293e+05,5.818592e+05,5.193904e+05,6.225533e+06,5.505067e+05,5.295484e+05,5.423697e+05,4.409502e+05,4.398330e+05,5.005290e+05,07120613
5,ADDISON,MT,390000029,6.227223e+05,6.721147e+05,6.530309e+05,6.509152e+05,6.989734e+05,6.314568e+05,7.668743e+06,6.741782e+05,6.469025e+05,6.657130e+05,5.568542e+05,5.647214e+05,6.311606e+05,07120613
6,ADDISON,MTART,390000029,7.957000e+01,6.164500e+02,4.946400e+02,6.330300e+02,7.411300e+02,3.551500e+02,4.824660e+03,4.921800e+02,3.026900e+02,3.148800e+02,3.693400e+02,2.080800e+02,2.175200e+02,07120613
7,ADELINE,MT,390000030,7.632000e+01,9.552000e+01,3.202000e+01,4.994000e+01,4.848000e+01,1.083300e+02,1.033020e+03,8.848000e+01,1.012800e+02,1.307300e+02,1.232000e+02,1.161600e+02,6.256000e+01,07120613
8,ALBANY,MT,390000020,4.279290e+03,3.849030e+03,4.750800e+03,5.070320e+03,3.507260e+03,3.996910e+03,4.445245e+04,2.474710e+03,2.374660e+03,3.195890e+03,3.783490e+03,3.128050e+03,4.042040e+03,07120613
9,ALBERS,MT,390000045,7.649310e+03,7.201670e+03,6.731610e+03,6.484990e+03,7.233930e+03,7.296550e+03,9.000683e+04,7.828690e+03,7.630750e+03,8.008930e+03,8.271500e+03,7.563530e+03,8.105370e+03,07120613


In [8]:
# x1 = top
# y1 = left
# x2 = top+height
# y2 = left+width
!java -jar tabula-1.0.2-jar-with-dependencies.jar -p all -a "50.0, 0.0, 600.0, 800.0" -o cordin.csv sales07060607.pdf

Error: Unable to access jarfile tabula-1.0.2-jar-with-dependencies.jar
